# Default functionality

In [1]:
library(data.table)

In [2]:
DT <- fread("melt_default.csv")

In [3]:
DT

family_id,age_mother,dob_child1,dob_child2,dob_child3
1,30,1998-11-26,2000-01-29,NA
2,27,1996-06-22,NA,NA
3,26,2002-07-11,2004-04-05,2007-09-02
4,32,2004-10-10,2009-08-27,2012-07-21
5,29,2000-12-05,2005-02-28,NA


In [4]:
str(DT)

Classes ‘data.table’ and 'data.frame':	5 obs. of  5 variables:
 $ family_id : int  1 2 3 4 5
 $ age_mother: int  30 27 26 32 29
 $ dob_child1: chr  "1998-11-26" "1996-06-22" "2002-07-11" "2004-10-10" ...
 $ dob_child2: chr  "2000-01-29" NA "2004-04-05" "2009-08-27" ...
 $ dob_child3: chr  NA NA "2007-09-02" "2012-07-21" ...
 - attr(*, ".internal.selfref")=<externalptr> 


Convert DT to long form where each dob is a separate observation.

In [5]:
DT.m1 = melt(DT, id.vars = c("family_id", "age_mother"),
                measure.vars = c("dob_child1", "dob_child2", "dob_child3"))

In [6]:
DT.m1

family_id,age_mother,variable,value
1,30,dob_child1,1998-11-26
2,27,dob_child1,1996-06-22
3,26,dob_child1,2002-07-11
4,32,dob_child1,2004-10-10
5,29,dob_child1,2000-12-05
1,30,dob_child2,2000-01-29
2,27,dob_child2,NA
3,26,dob_child2,2004-04-05
4,32,dob_child2,2009-08-27
5,29,dob_child2,2005-02-28


In [7]:
str(DT.m1)

Classes ‘data.table’ and 'data.frame':	15 obs. of  4 variables:
 $ family_id : int  1 2 3 4 5 1 2 3 4 5 ...
 $ age_mother: int  30 27 26 32 29 30 27 26 32 29 ...
 $ variable  : Factor w/ 3 levels "dob_child1","dob_child2",..: 1 1 1 1 1 2 2 2 2 2 ...
 $ value     : chr  "1998-11-26" "1996-06-22" "2002-07-11" "2004-10-10" ...
 - attr(*, ".internal.selfref")=<externalptr> 


- measure.vars specify the set of columns we would like to collapse (or combine) together.
- We can also specify column indices instead of names.
- By default, variable column is of type factor. Set variable.factor argument to FALSE if you'd like to return a character vector instead. variable.factor argument is only available in melt from data.table and not in the reshape2 package.
- By default, the molten columns are automatically named variable and value.
- melt preserves column attributes in result.


Name the variable and value columns to child and dob respectively

In [8]:
DT.m1 = melt(DT, measure.vars = c("dob_child1", "dob_child2", "dob_child3"),
               variable.name = "child", value.name = "dob")
DT.m1

family_id,age_mother,child,dob
1,30,dob_child1,1998-11-26
2,27,dob_child1,1996-06-22
3,26,dob_child1,2002-07-11
4,32,dob_child1,2004-10-10
5,29,dob_child1,2000-12-05
1,30,dob_child2,2000-01-29
2,27,dob_child2,NA
3,26,dob_child2,2004-04-05
4,32,dob_child2,2009-08-27
5,29,dob_child2,2005-02-28


That is, we'd like to collect all child observations corresponding to each family_id, age_mother together under the same row. We can accomplish it using dcast as follows:

In [9]:
dcast(DT.m1, family_id + age_mother ~ child, value.var = "dob")

family_id,age_mother,dob_child1,dob_child2,dob_child3
1,30,1998-11-26,2000-01-29,NA
2,27,1996-06-22,NA,NA
3,26,2002-07-11,2004-04-05,2007-09-02
4,32,2004-10-10,2009-08-27,2012-07-21
5,29,2000-12-05,2005-02-28,NA


In [14]:
namess <- DT.m1[,names(.SD)]
reformulate(namess[3], namess[1:2])

family_id ~ child

In [15]:
?reformulate

- dcast uses formula interface. The variables on the LHS of formula represents the id vars and RHS the measure vars.
- value.var denotes the column to be filled in with while casting to wide format.
- dcast also tries to preserve attributes in result wherever possible.


You can also pass a function to aggregate by in dcast with the argument fun.aggregate. This is particularly essential when the formula provided does not identify single observation for each cell.

In [10]:
dcast(DT.m1, family_id ~ ., fun.agg = function(x) sum(!is.na(x)), value.var = "dob")

family_id,.
1,2
2,1
3,3
4,3
5,2


# Limitations

In [17]:
DT <- fread("melt_enhanced.csv")
DT

family_id,age_mother,dob_child1,dob_child2,dob_child3,gender_child1,gender_child2,gender_child3
1,30,1998-11-26,2000-01-29,NA,1,2,NA
2,27,1996-06-22,NA,NA,2,NA,NA
3,26,2002-07-11,2004-04-05,2007-09-02,2,2,1
4,32,2004-10-10,2009-08-27,2012-07-21,1,1,1
5,29,2000-12-05,2005-02-28,NA,2,1,NA


And you'd like to combine (melt) all the dob columns together, and gender columns together. Using the current functionality, we can do something like this:

In [21]:
DT.m1 <- melt(DT, id = c("family_id", "age_mother"))
DT.m1[, c("variable", "child") := tstrsplit(variable, "_", fixed = TRUE)]
DT.c1 = dcast(DT.m1, family_id + age_mother + child ~ variable, value.var = "value")
DT.c1

Warning message in melt.data.table(DT, id = c("family_id", "age_mother")):
“'measure.vars' [dob_child1, dob_child2, dob_child3, gender_child1, ...] are not all of the same type. By order of hierarchy, the molten data value column will be of type 'character'. All measure variables not of type 'character' will be coerced to. Check DETAILS in ?melt.data.table for more on coercion.”

family_id,age_mother,child,dob,gender
1,30,child1,1998-11-26,1
1,30,child2,2000-01-29,2
1,30,child3,NA,NA
2,27,child1,1996-06-22,2
2,27,child2,NA,NA
2,27,child3,NA,NA
3,26,child1,2002-07-11,2
3,26,child2,2004-04-05,2
3,26,child3,2007-09-02,1
4,32,child1,2004-10-10,1


# Enhanced functionality

## Enhanced melt

Since we'd like for data.tables to perform this operation straightforward and efficient using the same interface, we went ahead and implemented an additional functionality, where we can melt to multiple columns simultaneously.

In [26]:
DT
colA = paste("dob_child", 1:3, sep = "")
colB = paste("gender_child", 1:3, sep = "")
DT.m2 = melt(DT, measure = list(colA, colB), value.name = c("dob", "gender"))
DT.m2

family_id,age_mother,dob_child1,dob_child2,dob_child3,gender_child1,gender_child2,gender_child3
1,30,1998-11-26,2000-01-29,NA,1,2,NA
2,27,1996-06-22,NA,NA,2,NA,NA
3,26,2002-07-11,2004-04-05,2007-09-02,2,2,1
4,32,2004-10-10,2009-08-27,2012-07-21,1,1,1
5,29,2000-12-05,2005-02-28,NA,2,1,NA


family_id,age_mother,variable,dob,gender
1,30,1,1998-11-26,1
2,27,1,1996-06-22,2
3,26,1,2002-07-11,2
4,32,1,2004-10-10,1
5,29,1,2000-12-05,2
1,30,2,2000-01-29,2
2,27,2,NA,NA
3,26,2,2004-04-05,2
4,32,2,2009-08-27,1
5,29,2,2005-02-28,1


## Enhanced dcast

Okay great! We can now melt into multiple columns simultaneously. Now given the data set DT.m2 as shown above, how can we get back to the same format as the original data we started with?

If we use the current functionality of dcast, then we'd have to cast twice and bind the results together. But that's once again verbose, not straightforward and is also inefficient.

In [28]:
DT.m2
DT.c2 = dcast(DT.m2, family_id + age_mother ~ variable, value.var = c("dob", "gender"))
DT.c2

family_id,age_mother,variable,dob,gender
1,30,1,1998-11-26,1
2,27,1,1996-06-22,2
3,26,1,2002-07-11,2
4,32,1,2004-10-10,1
5,29,1,2000-12-05,2
1,30,2,2000-01-29,2
2,27,2,NA,NA
3,26,2,2004-04-05,2
4,32,2,2009-08-27,1
5,29,2,2005-02-28,1


family_id,age_mother,dob_1,dob_2,dob_3,gender_1,gender_2,gender_3
1,30,1998-11-26,2000-01-29,NA,1,2,NA
2,27,1996-06-22,NA,NA,2,NA,NA
3,26,2002-07-11,2004-04-05,2007-09-02,2,2,1
4,32,2004-10-10,2009-08-27,2012-07-21,1,1,1
5,29,2000-12-05,2005-02-28,NA,2,1,NA


Multiple functions to fun.aggregate: {.bs-callout .bs-callout-info}

You can also provide multiple functions to fun.aggregate to dcast for data.tables. Check the examples in ?dcast which illustrates this functionality.